# Mortgage Forecasting - Final Forecasting & Evaluation
## Production Forecasting and Business Insights

**Chain of Thought:**
1. Load best model → 2. Retrain on full data → 3. Generate future forecasts → 4. Calculate confidence intervals → 5. Create business visualizations → 6. Generate executive insights

**Business Context:** We now deploy our selected model to generate actual forecasts that can support business planning, resource allocation, and strategic decision-making.

## 1. Load Best Model and Full Dataset

**Thinking Process:** We start by loading our selected best-performing model and the complete historical dataset. We'll retrain on all available data to maximize forecast accuracy.

In [ ]:
# === INITIALIZATION AND IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from prophet.serialize import model_from_json
import yaml
import json
import warnings
warnings.filterwarnings('ignore')

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ Forecasting libraries imported successfully")

In [ ]:
# === LOAD CONFIGURATION AND MODEL RESULTS ===
with open('../config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Load model evaluation results
with open('../outputs/model_metadata.json', 'r') as f:
    model_metadata = json.load(f)

# Load enhanced data
data_path = "../data/processed/quarterly_data_enhanced.csv"
full_data = pd.read_csv(data_path, parse_dates=['date'])

# Prepare full time series
ts_full = full_data.set_index('date')['total_loan_volume']
ts_full = ts_full.asfreq('Q')

print("🔧 PRODUCTION FORECASTING SETUP")
print("=" * 50)
print(f"Geography: {config['data']['target_geography']}")
print(f"Best Model: {model_metadata['best_model']}")
print(f"Model Accuracy: {model_metadata['best_accuracy']:.1f}%")
print(f"Full Data Period: {ts_full.index.min().date()} to {ts_full.index.max().date()}")
print(f"Total Historical Quarters: {len(ts_full)}")
print(f"Forecast Horizon: {config['forecasting']['final_forecast_quarters']} quarters")

# Display recent data trends
recent_growth = (ts_full.iloc[-1] / ts_full.iloc[-5] - 1) * 100  # Last year growth
print(f"\n📈 RECENT TRENDS:")
print(f"   Recent YoY Growth: {recent_growth:+.1f}%")
print(f"   Current Quarter: ${ts_full.iloc[-1]/1e6:.1f}M")
print(f"   Volatility (Last 2 years): {(ts_full[-8:].std() / ts_full[-8:].mean())*100:.1f}%")

## 2. Load and Retrain Best Model

**Chain of Thought:** We retrain our best model on the complete historical dataset to incorporate all available information for our final forecasts.

In [ ]:
# === LOAD AND RETRAIN BEST MODEL ===
def load_and_retrain_best_model(model_name, full_data):
    """
    Load the best performing model and retrain on full dataset
    
    THINKING: Retraining on full data:
    - Incorporates all available information
    - Improves parameter estimation
    - Provides most robust forecasts
    """
    
    print(f"\n🔄 RETRAINING {model_name} ON FULL DATASET")
    print("=" * 50)
    
    if model_name == 'SARIMA':
        # Load the original model to get optimal parameters
        original_model = joblib.load('../models/sarima_model.pkl')
        
        print("📊 Original SARIMA parameters:")
        print(f"   Order: {original_model.order}")
        print(f"   Seasonal Order: {original_model.seasonal_order}")
        
        # Retrain with same parameters on full data
        from pmdarima import ARIMA
        final_model = ARIMA(
            order=original_model.order,
            seasonal_order=original_model.seasonal_order,
            suppress_warnings=True
        )
        final_model.fit(full_data)
        
        print("✅ SARIMA retrained on full data")
        
    elif model_name == 'Prophet':
        # Load and retrain Prophet
        with open('../models/prophet_model.json', 'r') as f:
            final_model = model_from_json(f.read())
        
        # Prepare full data for Prophet
        prophet_data = full_data.reset_index()
        prophet_data.columns = ['ds', 'y']
        
        # Retrain on full data
        final_model.fit(prophet_data)
        
        print("✅ Prophet retrained on full data")
        
    elif model_name == 'ETS':
        # Load original model structure and retrain
        from statsmodels.tsa.holtwinters import ExponentialSmoothing
        
        # Use configuration from EDA insights
        with open('../outputs/eda_results.json', 'r') as f:
            eda_insights = json.load(f)
        
        has_seasonality = eda_insights['pattern_analysis']['seasonal_strength'] > 0.3
        
        if has_seasonality:
            final_model = ExponentialSmoothing(
                full_data,
                seasonal_periods=4,
                trend='add',
                seasonal='add'
            ).fit()
        else:
            final_model = ExponentialSmoothing(full_data, trend='add').fit()
        
        print("✅ ETS retrained on full data")
    
    else:
        # For baseline models, we'll use simple approaches
        print(f"⚠️  Using simple forecasting for {model_name}")
        final_model = None
    
    return final_model

In [ ]:
# === RETRAIN BEST MODEL ===
best_model_name = model_metadata['best_model']
final_model = load_and_retrain_best_model(best_model_name, ts_full)

print(f"\n✅ BEST MODEL READY FOR PRODUCTION FORECASTING")
print(f"   Model: {best_model_name}")
print(f"   Historical Accuracy: {model_metadata['best_accuracy']:.1f}%")
print(f"   Training Data: {len(ts_full)} quarters")
print(f"   Forecast Horizon: {config['forecasting']['final_forecast_quarters']} quarters")

## 3. Generate Future Forecasts

**Thinking Process:** We generate forecasts for the specified horizon, ensuring we handle each model type appropriately and create proper future dates.

In [ ]:
# === FORECAST GENERATION FUNCTION ===
def generate_production_forecasts(model, model_name, historical_data, periods=8):
    """
    Generate production forecasts using the best model
    
    THINKING: Production forecasting considerations:
    - Proper future date alignment
    - Model-specific forecast methods
    - Handling of confidence intervals
    - Business-ready output format
    """
    
    print(f"\n📈 GENERATING {periods}-QUARTER PRODUCTION FORECAST")
    print("=" * 50)
    
    if model_name == 'SARIMA':
        # SARIMA forecast
        forecast_values, conf_int = model.predict(
            n_periods=periods, 
            return_conf_int=True,
            alpha=1-config['forecasting']['confidence_level']
        )
        
    elif model_name == 'Prophet':
        # Prophet forecast
        future = model.make_future_dataframe(
            periods=periods, 
            freq='Q', 
            include_history=False
        )
        forecast_df = model.predict(future)
        forecast_values = forecast_df['yhat'].values
        
        # Prophet provides confidence intervals
        conf_int = np.column_stack([
            forecast_df['yhat_lower'].values,
            forecast_df['yhat_upper'].values
        ])
        
    elif model_name == 'ETS':
        # ETS forecast
        forecast_values = model.forecast(periods)
        
        # Simple confidence interval for ETS
        resid_std = np.std(model.resid)
        conf_int = np.column_stack([
            forecast_values - 1.96 * resid_std,
            forecast_values + 1.96 * resid_std
        ])
    
    else:
        # Simple models (naive, seasonal naive, etc.)
        if model_name == 'Naive':
            forecast_values = [historical_data.iloc[-1]] * periods
        elif model_name == 'Seasonal_Naive':
            # Use last year's pattern
            forecast_values = []
            for i in range(periods):
                lookback = min(4, len(historical_data))
                forecast_values.append(historical_data.iloc[-lookback + (i % 4)])
        else:
            # Moving average fallback
            avg_value = historical_data.mean()
            forecast_values = [avg_value] * periods
        
        forecast_values = np.array(forecast_values)
        
        # Simple confidence interval
        historical_std = historical_data.std()
        conf_int = np.column_stack([
            forecast_values - 1.96 * historical_std,
            forecast_values + 1.96 * historical_std
        ])
    
    # Create future dates
    last_date = historical_data.index.max()
    future_dates = pd.date_range(
        start=last_date + pd.DateOffset(months=3),
        periods=periods,
        freq='Q'
    )
    
    # Create forecast series
    forecast_series = pd.Series(forecast_values, index=future_dates)
    
    # Create confidence interval dataframe
    conf_int_df = pd.DataFrame({
        'lower': conf_int[:, 0],
        'upper': conf_int[:, 1]
    }, index=future_dates)
    
    print(f"✅ PRODUCTION FORECAST GENERATED")
    print(f"   Forecast Period: {future_dates.min().date()} to {future_dates.max().date()}")
    print(f"   Mean Forecast: ${forecast_series.mean()/1e6:.1f}M per quarter")
    print(f"   Total Forecast Volume: ${forecast_series.sum()/1e9:.2f}B")
    print(f"   Confidence Level: {config['forecasting']['confidence_level']*100:.0f}%")
    
    return forecast_series, conf_int_df

In [ ]:
# === GENERATE PRODUCTION FORECASTS ===
forecast_horizon = config['forecasting']['final_forecast_quarters']
future_forecast, confidence_intervals = generate_production_forecasts(
    final_model, best_model_name, ts_full, periods=forecast_horizon
)

In [ ]:
# === DISPLAY FORECAST RESULTS ===
print("\n📊 DETAILED FORECAST RESULTS")
print("=" * 50)

# Create business-ready forecast table
forecast_table = pd.DataFrame({
    'Quarter': future_forecast.index.strftime('%Y-Q%q'),
    'Point_Forecast': future_forecast.values,
    'Forecast_Millions': (future_forecast.values / 1e6).round(1),
    'Lower_CI_Millions': (confidence_intervals['lower'].values / 1e6).round(1),
    'Upper_CI_Millions': (confidence_intervals['upper'].values / 1e6).round(1),
    'Confidence_Range_Millions': ((confidence_intervals['upper'] - confidence_intervals['lower']).values / 1e6).round(1)
})

print("🔢 QUARTERLY FORECAST BREAKDOWN:")
display(forecast_table)

# Calculate key forecast metrics
forecast_growth = (future_forecast.iloc[-1] / ts_full.iloc[-1] - 1) * 100
avg_forecast = future_forecast.mean()
forecast_volatility = future_forecast.std() / future_forecast.mean() * 100

print(f"\n📈 FORECAST KEY METRICS:")
print(f"   Average Quarterly Forecast: ${avg_forecast/1e6:.1f}M")
print(f"   End-of-Horizon Growth: {forecast_growth:+.1f}% vs current")
print(f"   Forecast Volatility: {forecast_volatility:.1f}%")
print(f"   Confidence Range: ±${((confidence_intervals['upper'] - confidence_intervals['lower']).mean() / 2 / 1e6):.1f}M")

## 4. Create Production Visualization

**Chain of Thought:** We need executive-ready visualizations that clearly communicate the forecast results and uncertainty to business stakeholders.

In [ ]:
# === PRODUCTION VISUALIZATION FUNCTION ===
def create_production_visualization(historical_data, forecast_series, conf_int_df, 
                                  model_name, accuracy, geography):
    """
    Create executive-ready production forecast visualization
    
    THINKING: Executive visualization requirements:
    - Clear distinction between historical and forecast
    - Confidence intervals for uncertainty communication
    - Professional, business-appropriate styling
    - Key metrics highlighted
    """
    
    print("\n🎨 CREATING EXECUTIVE VISUALIZATION")
    print("=" * 50)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # === PLOT 1: HISTORICAL + FORECAST ===
    # Historical data
    ax1.plot(historical_data.index, historical_data.values / 1e6, 
             label='Historical Data', color='#2E86AB', linewidth=3, alpha=0.8)
    
    # Forecast
    forecast_dates = forecast_series.index
    ax1.plot(forecast_dates, forecast_series.values / 1e6, 
             label='Forecast', color='#D63230', linewidth=3, linestyle='-', marker='o')
    
    # Confidence interval
    ax1.fill_between(forecast_dates, 
                    conf_int_df['lower'] / 1e6, 
                    conf_int_df['upper'] / 1e6,
                    alpha=0.3, color='#D63230', 
                    label=f'{config["forecasting"]["confidence_level"]*100:.0f}% Confidence Interval')
    
    # Add separation line and annotation
    last_historical_date = historical_data.index.max()
    ax1.axvline(x=last_historical_date, color='gray', linestyle=':', alpha=0.7, linewidth=2)
    ax1.text(last_historical_date, ax1.get_ylim()[1] * 0.9, 'Forecast Start', 
             rotation=90, va='top', ha='right', fontsize=11, fontweight='bold')
    
    # Formatting
    ax1.set_title(
        f'Mortgage Origination Forecast: {geography}\n'
        f'{model_name} Model | Historical Accuracy: {accuracy:.1f}%',
        fontsize=16, fontweight='bold', pad=20
    )
    ax1.set_ylabel('Volume (Millions $)', fontsize=12, fontweight='bold')
    ax1.legend(fontsize=11, loc='upper left')
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Add forecast statistics annotation
    avg_forecast = forecast_series.mean() / 1e6
    total_forecast = forecast_series.sum() / 1e9
    
    stats_text = f'Forecast Summary:\n• Avg Quarterly: ${avg_forecast:.1f}M\n• Total: ${total_forecast:.1f}B'
    ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
             fontsize=10, fontweight='bold')
    
    # === PLOT 2: QUARTERLY GROWTH FORECAST ===
    # Calculate quarter-over-quarter growth
    combined_series = pd.concat([historical_data.iloc[-1:], forecast_series])
    qoq_growth = combined_series.pct_change().iloc[1:] * 100
    
    # Calculate YoY growth (4-quarter difference)
    if len(forecast_series) >= 4:
        # Combine last year of historical with forecast
        recent_historical = historical_data.iloc[-4:]
        extended_series = pd.concat([recent_historical, forecast_series])
        yoy_growth = extended_series.pct_change(4).iloc[4:] * 100
    else:
        yoy_growth = pd.Series(dtype=float)
    
    # Plot growth rates
    quarters = [f'Q{(i%4)+1}\n{date.year}' for i, date in enumerate(forecast_series.index)]
    
    if not qoq_growth.empty:
        bars1 = ax2.bar(np.arange(len(qoq_growth)) - 0.2, qoq_growth.values, 
                       0.4, label='Quarter-over-Quarter', alpha=0.8, color='#4ECDC4')
    
    if not yoy_growth.empty:
        bars2 = ax2.bar(np.arange(len(yoy_growth)) + 0.2, yoy_growth.values, 
                       0.4, label='Year-over-Year', alpha=0.8, color='#45B7D1')
    
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    ax2.set_title('Forecasted Growth Rates', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Growth Rate (%)', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Forecast Quarter', fontsize=12, fontweight='bold')
    
    if len(forecast_series) <= 8:
        ax2.set_xticks(np.arange(len(forecast_series)))
        ax2.set_xticklabels(quarters, rotation=45)
    
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Add value labels on bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + (0.3 if height >= 0 else -0.8),
                    f'{height:.1f}%', ha='center', va='bottom' if height >= 0 else 'top', 
                    fontweight='bold', fontsize=9)
    
    if not qoq_growth.empty:
        add_value_labels(bars1)
    if not yoy_growth.empty:
        add_value_labels(bars2)
    
    plt.tight_layout()
    plt.show()
    
    # Save the production visualization
    fig.savefig('../outputs/production_forecast_visualization.png', dpi=300, bbox_inches='tight')
    print("✅ Executive visualization saved: ../outputs/production_forecast_visualization.png")
    
    return fig

In [ ]:
# === CREATE PRODUCTION VISUALIZATION ===
production_viz = create_production_visualization(
    ts_full, future_forecast, confidence_intervals, 
    best_model_name, model_metadata['best_accuracy'], 
    config['data']['target_geography']
)

## 5. Business Insights and Recommendations

**Thinking Process:** We translate our statistical forecasts into actionable business insights that support decision-making across different departments.

In [ ]:
# === BUSINESS INSIGHTS GENERATION ===
def generate_business_insights(historical_data, forecast_series, conf_int_df, model_name, accuracy):
    """
    Generate actionable business insights from forecasts
    
    THINKING: Different stakeholders need different insights:
    - Executives: Strategic direction and growth
    - Operations: Resource planning and capacity
    - Risk: Uncertainty and downside scenarios
    - Finance: Revenue projections and planning
    """
    
    print("\n" + "=" * 70)
    print("BUSINESS INSIGHTS & STRATEGIC RECOMMENDATIONS")
    print("=" * 70)
    
    # Calculate key business metrics
    current_volume = historical_data.iloc[-1]
    avg_forecast = forecast_series.mean()
    forecast_growth = (avg_forecast / current_volume - 1) * 100
    total_forecast_volume = forecast_series.sum()
    
    # Risk assessment
    downside_risk = (conf_int_df['lower'].mean() / current_volume - 1) * 100
    upside_potential = (conf_int_df['upper'].mean() / current_volume - 1) * 100
    uncertainty_range = (conf_int_df['upper'] - conf_int_df['lower']).mean() / current_volume * 100
    
    # Seasonal pattern analysis
    seasonal_pattern = forecast_series.groupby(forecast_series.index.quarter).mean()
    peak_quarter = seasonal_pattern.idxmax()
    trough_quarter = seasonal_pattern.idxmin()
    seasonal_amplitude = (seasonal_pattern.max() / seasonal_pattern.min() - 1) * 100
    
    print(f"\n📈 EXECUTIVE SUMMARY:")
    print(f"   • Forecast Model: {model_name} ({accuracy:.1f}% historical accuracy)")
    print(f"   • Forecast Horizon: {len(forecast_series)} quarters")
    print(f"   • Expected Growth: {forecast_growth:+.1f}% vs current levels")
    print(f"   • Total Projected Volume: ${total_forecast_volume/1e9:.2f}B")
    
    print(f"\n🎯 STRATEGIC INSIGHTS:")
    
    if forecast_growth > 10:
        print("   • 🚀 STRONG GROWTH: Market expansion opportunities")
        print("   • Consider capacity expansion and market penetration")
    elif forecast_growth > 5:
        print("   • 📈 MODERATE GROWTH: Stable market conditions")
        print("   • Focus on operational efficiency and market share")
    elif forecast_growth > 0:
        print("   • 📊 MARGINAL GROWTH: Competitive market")
        print("   • Emphasize customer retention and cost optimization")
    else:
        print("   • 📉 MARKET CONTRACTION: Defensive strategy needed")
        print("   • Focus on risk management and operational flexibility")
    
    print(f"\n📅 SEASONAL PATTERN ANALYSIS:")
    print(f"   • Peak Quarter: Q{peak_quarter} (${seasonal_pattern.max()/1e6:.1f}M)")
    print(f"   • Trough Quarter: Q{trough_quarter} (${seasonal_pattern.min()/1e6:.1f}M)")
    print(f"   • Seasonal Amplitude: {seasonal_amplitude:.1f}%")
    
    if seasonal_amplitude > 20:
        print("   • 🎯 Strong seasonality - plan for capacity fluctuations")
    elif seasonal_amplitude > 10:
        print("   • 📊 Moderate seasonality - standard seasonal planning")
    else:
        print("   • 📈 Minimal seasonality - stable operations possible")
    
    print(f"\n⚖️  RISK ASSESSMENT:")
    print(f"   • Downside Risk: {downside_risk:+.1f}% vs current")
    print(f"   • Upside Potential: {upside_potential:+.1f}% vs current")
    print(f"   • Uncertainty Range: ±{uncertainty_range/2:.1f}%")
    
    if uncertainty_range > 30:
        print("   • 🎢 HIGH UNCERTAINTY: Maintain flexible operations")
    elif uncertainty_range > 20:
        print("   • ⚠️  MODERATE UNCERTAINTY: Standard contingency planning")
    else:
        print("   • ✅ LOW UNCERTAINTY: Confident planning possible")
    
    print(f"\n💼 DEPARTMENTAL RECOMMENDATIONS:")
    print("   • OPERATIONS: Plan for {} capacity vs current"
          .format("increased" if forecast_growth > 5 else "stable" if forecast_growth > -5 else "reduced"))
    print("   • FINANCE: Project {} revenue streams"
          .format("growing" if forecast_growth > 3 else "stable" if forecast_growth > -3 else "declining"))
    print("   • RISK: {} risk exposure given forecast uncertainty"
          .format("Monitor" if uncertainty_range > 25 else "Standard monitoring of"))
    print("   • STRATEGY: {} market position"
          .format("Expand" if forecast_growth > 8 else "Maintain" if forecast_growth > 0 else "Defend"))
    
    print(f"\n📋 QUARTERLY ACTION PLAN:")
    for i, (date, point_fcst) in enumerate(forecast_series.items(), 1):
        lower_bound = conf_int_df.loc[date, 'lower'] / 1e6
        upper_bound = conf_int_df.loc[date, 'upper'] / 1e6
        
        print(f"   {i}. {date.strftime('%Y-Q%q')}: ${point_fcst/1e6:.1f}M (${lower_bound:.1f}M - ${upper_bound:.1f}M)")
    
    print("\n" + "=" * 70)

In [ ]:
# === GENERATE BUSINESS INSIGHTS ===
generate_business_insights(
    ts_full, future_forecast, confidence_intervals, 
    best_model_name, model_metadata['best_accuracy']
)

## 6. Export Final Results

**Chain of Thought:** We create multiple output formats to serve different stakeholders - from detailed technical reports to executive summaries.

In [ ]:
# === COMPREHENSIVE RESULTS EXPORT ===
def export_final_results(historical_data, forecast_series, conf_int_df, model_name, accuracy):
    """
    Export final results in multiple formats for different stakeholders
    
    THINKING: Different stakeholders need different information:
    - Technical team: Detailed forecasts with confidence intervals
    - Business users: Simplified tables and visualizations  
    - Executives: High-level summary and recommendations
    """
    
    print("\n💾 EXPORTING FINAL RESULTS")
    print("=" * 50)
    
    from pathlib import Path
    
    # Ensure outputs directory exists
    Path("../outputs").mkdir(exist_ok=True)
    
    # === 1. DETAILED FORECAST DATA (Technical) ===
    detailed_results = []
    
    # Historical data
    for date, value in historical_data.items():
        detailed_results.append({
            'period': date.strftime('%Y-Q%q'),
            'date': date,
            'type': 'historical',
            'volume': value,
            'volume_millions': value / 1e6,
            'lower_ci': np.nan,
            'upper_ci': np.nan,
            'model': 'N/A'
        })
    
    # Forecast data
    for date in forecast_series.index:
        detailed_results.append({
            'period': date.strftime('%Y-Q%q'),
            'date': date,
            'type': 'forecast',
            'volume': forecast_series[date],
            'volume_millions': forecast_series[date] / 1e6,
            'lower_ci': conf_int_df.loc[date, 'lower'],
            'upper_ci': conf_int_df.loc[date, 'upper'],
            'model': model_name
        })
    
    detailed_df = pd.DataFrame(detailed_results)
    detailed_df.to_csv('../outputs/detailed_forecast_results.csv', index=False)
    print("✅ Detailed forecast data: ../outputs/detailed_forecast_results.csv")
    
    # === 2. EXECUTIVE SUMMARY (Business) ===
    executive_summary = {
        'project_title': 'Regional Mortgage Origination Forecasting',
        'geography': config['data']['target_geography'],
        'forecast_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
        'key_metrics': {
            'selected_model': model_name,
            'historical_accuracy': f"{accuracy:.1f}%",
            'forecast_horizon': f"{len(forecast_series)} quarters",
            'confidence_level': f"{config['forecasting']['confidence_level']*100:.0f}%",
            'current_volume': f"${historical_data.iloc[-1]/1e6:.1f}M",
            'average_forecast': f"${forecast_series.mean()/1e6:.1f}M",
            'total_forecast_volume': f"${forecast_series.sum()/1e9:.2f}B",
            'expected_growth': f"{(forecast_series.mean() / historical_data.iloc[-1] - 1)*100:+.1f}%"
        },
        'forecast_period': {
            'start': forecast_series.index.min().strftime('%Y-%m-%d'),
            'end': forecast_series.index.max().strftime('%Y-%m-%d')
        },
        'risk_assessment': {
            'uncertainty_level': 'High' if (conf_int_df['upper'] - conf_int_df['lower']).mean() / forecast_series.mean() > 0.3 
                               else 'Medium' if (conf_int_df['upper'] - conf_int_df['lower']).mean() / forecast_series.mean() > 0.2 
                               else 'Low',
            'downside_risk': f"{(conf_int_df['lower'].mean() / historical_data.iloc[-1] - 1)*100:+.1f}%",
            'upside_potential': f"{(conf_int_df['upper'].mean() / historical_data.iloc[-1] - 1)*100:+.1f}%"
        }
    }
    
    with open('../outputs/executive_summary.json', 'w') as f:
        json.dump(executive_summary, f, indent=2)
    
    print("✅ Executive summary: ../outputs/executive_summary.json")
    
    # === 3. QUARTERLY FORECAST TABLE (Operations) ===
    quarterly_forecast = pd.DataFrame({
        'Quarter': forecast_series.index.strftime('%Y-Q%q'),
        'Point_Forecast_Millions': (forecast_series.values / 1e6).round(1),
        'Lower_Bound_Millions': (conf_int_df['lower'].values / 1e6).round(1),
        'Upper_Bound_Millions': (conf_int_df['upper'].values / 1e6).round(1),
        'QoQ_Growth_Pct': (forecast_series.pct_change().fillna(0) * 100).round(1)
    })
    
    quarterly_forecast.to_csv('../outputs/quarterly_forecast_table.csv', index=False)
    print("✅ Quarterly forecast table: ../outputs/quarterly_forecast_table.csv")
    
    # === 4. MODEL PERFORMANCE REPORT (Technical) ===
    performance_report = {
        'model_performance': {
            'final_model': model_name,
            'historical_accuracy': accuracy,
            'forecast_generated': pd.Timestamp.now().isoformat(),
            'data_points_used': len(historical_data),
            'forecast_horizon': len(forecast_series)
        },
        'data_characteristics': {
            'date_range': {
                'start': historical_data.index.min().isoformat(),
                'end': historical_data.index.max().isoformat()
            },
            'total_volume': historical_data.sum(),
            'average_volume': historical_data.mean(),
            'volatility': (historical_data.std() / historical_data.mean() * 100)
        },
        'forecast_characteristics': {
            'average_forecast': forecast_series.mean(),
            'forecast_volatility': (forecast_series.std() / forecast_series.mean() * 100),
            'confidence_interval_width': (conf_int_df['upper'] - conf_int_df['lower']).mean()
        }
    }
    
    with open('../outputs/model_performance_report.json', 'w') as f:
        json.dump(performance_report, f, indent=2)
    
    print("✅ Model performance report: ../outputs/model_performance_report.json")
    
    print(f"\n📁 ALL RESULTS EXPORTED SUCCESSFULLY!")
    
    return detailed_df, executive_summary, quarterly_forecast

In [ ]:
# === EXPORT ALL FINAL RESULTS ===
detailed_results, executive_summary, quarterly_forecast = export_final_results(
    ts_full, future_forecast, confidence_intervals, 
    best_model_name, model_metadata['best_accuracy']
)

## 7. Project Completion Summary

**Thinking Process:** We conclude with a comprehensive summary of the entire project, highlighting achievements, limitations, and next steps for continuous improvement.

In [ ]:
# === FINAL PROJECT SUMMARY ===
print("\n" + "=" * 70)
print("PROJECT COMPLETION SUMMARY")
print("=" * 70)

print(f"\n🎯 PROJECT OBJECTIVES ACHIEVED:")
print("   ✅ Developed robust mortgage origination forecasting pipeline")
print("   ✅ Processed and validated multi-year HMDA data")
print("   ✅ Conducted comprehensive exploratory data analysis")
print("   ✅ Built and compared multiple forecasting models")
print("   ✅ Selected optimal model based on performance")
print("   ✅ Generated production-ready forecasts with confidence intervals")
print("   ✅ Delivered actionable business insights and recommendations")

print(f"\n📊 KEY DELIVERABLES PRODUCED:")
print("   • Cleaned and processed time series data")
print("   • Comprehensive EDA reports and visualizations")
print("   • Multiple trained forecasting models")
print("   • Model performance evaluation and comparison")
print("   • Production forecasts with confidence intervals")
print("   • Executive summary and business recommendations")
print("   • Technical documentation and performance reports")

print(f"\n🔧 TECHNICAL ACHIEVEMENTS:")
print(f"   • Best Model: {best_model_name}")
print(f"   • Forecast Accuracy: {model_metadata['best_accuracy']:.1f}%")
print(f"   • Forecast Horizon: {len(future_forecast)} quarters")
print(f"   • Confidence Level: {config['forecasting']['confidence_level']*100:.0f}%")
print(f"   • Data Processed: {len(ts_full)} historical quarters")

print(f"\n💼 BUSINESS IMPACT:")
print("   • Enables data-driven strategic planning")
print("   • Supports resource allocation and capacity planning")
print("   • Provides risk assessment with confidence intervals")
print("   • Facilitates market trend analysis and opportunity identification")
print("   • Enhances decision-making with quantitative forecasts")

print(f"\n🚀 PRODUCTION READINESS:")
print("   • Models trained and validated: ✅")
print("   • Forecasts generated with confidence intervals: ✅")
print("   • Business insights delivered: ✅")
print("   • Documentation complete: ✅")
print("   • Results exported in multiple formats: ✅")

print(f"\n📈 FORECAST OUTLOOK:")
avg_growth = (future_forecast.mean() / ts_full.iloc[-1] - 1) * 100
print(f"   • Expected Growth: {avg_growth:+.1f}% vs current levels")
print(f"   • Market Direction: {'Expanding' if avg_growth > 5 else 'Stable' if avg_growth > 0 else 'Contracting'}")
print(f"   • Planning Confidence: {'High' if model_metadata['best_accuracy'] >= 90 else 'Medium' if model_metadata['best_accuracy'] >= 85 else 'Moderate'}")

print(f"\n🔮 FUTURE ENHANCEMENTS:")
print("   • Incorporate external economic indicators")
print("   • Implement automated model retraining")
print("   • Develop real-time forecasting dashboard")
print("   • Expand to additional geographic regions")
print("   • Add scenario analysis and stress testing")

print(f"\n🎉 PROJECT SUCCESSFULLY COMPLETED!")
print("\n" + "=" * 70)
print("END OF NOTEBOOK 4: FORECASTING & EVALUATION")
print("=" * 70)